<a href="https://colab.research.google.com/github/cagBRT/Machine-Learning/blob/master/HyperParameterTuning_DNN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install --upgrade "jax[cpu]"

In [ ]:
!pip install scikeras

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data'
col_names = ['id','ri','na','mg','al','si','k','ca','ba','fe','glass_type']
glass = pd.read_csv(url, names=col_names, index_col='id')
glass.sort_values('al', inplace=True)
glass.head()

In [ ]:
# types 1, 2, 3 are window glass
# types 5, 6, 7 are household glass
glass['household'] = glass.glass_type.map({1:0, 2:0, 3:0, 5:1, 6:1, 7:1})
glass.head()

In [ ]:
feature_cols = ['ri','na','mg','al','si','k','ca','ba','fe','glass_type']
X = glass[feature_cols]
y = glass['household']

In [ ]:
X.shape

In [ ]:
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_shape=(10,), activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [ ]:
# create model
model = KerasClassifier(model=create_model, verbose=0,epochs=10,batch_size=20)

In [ ]:
model.fit(X,y)
predictions=model.predict(X)
score=model.score(X,y)
print(score)

# **Grid Search**

In [ ]:
# define the grid search parameters
batch_size = [5,10,20]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# **Random Search**

In [ ]:
random = RandomizedSearchCV(model, param_grid,n_iter=9)

In [ ]:
random.fit(X,y)

In [ ]:
print("Tuned Logistic Regression Parameters: {}".format(random.best_params_))
print("Best score is {}".format(random.best_score_))